# Assignment 4: Finite-Time Optimal Control

University of California Berkeley

ME C231A, EE C220B, Experiential Advanced Control I

***

These notes were developed by Roya Firoozi and Francesco Borrelli at UC Berkeley. They are protected by U.S. copyright law and by University policy (https://copyright.universityofcalifornia.edu/resources/ownership-course-materials.html).

If you are enrolled in ME C231A/EE C220B you may take notes and make copies of course materials for your own use. You may also share those materials with another student who is registered and enrolled in this course, and with DSP.

You may not reproduce, distribute or display (post/upload) (Links to an external site.) lecture notes or recordings or course materials in any other way — whether or not a fee is charged — without my express written consent. You also may not allow others to do so. If you do so, you may be subject to student conduct proceedings under the Links to an external site.Berkeley Code of Student Conduct, including Sections 102.23 and 102.25.



***

**NOTE**: Remove `...` in cells throughout this Colab note, and fill in your code.

In [11]:
# Run this cell only if you are using Google Colab. 

# install required dependencies
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !pip install -q pyomo
  !apt-get install -y -qq glpk-utils
  !apt-get install -y -qq coinor-cbc
  !wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
  !unzip -o -q ipopt-linux64
  !pip install ttictoc

***

# <font color=blue> 1. Finite-Time Optimal Control of a Vehicle </font>

In this question you solve a nonlinear finit-time optimal control problem, which is similar to the unicycle example discussed in class. For both parts of the question (part (a) and part (b)), please print out 4 subplots (one for each state vs time). Also print a simple plot showing the vehicle motion ($x_k$ vs $y_k$). In addition, print out 2 subplots (one for each control input vs time). Also, turn in the code for both parts (a) and (b), which formulate the optimization problems described below. For both parts, specify IPOPT as the solver in $\texttt{pyomo}$. 

Consider the same simplified kinematic bicycle model used in Homework 1.

\begin{eqnarray}
\dot{x} &=& v\cos(\psi+\beta)\nonumber\\
\dot{y} &=& v\sin(\psi+\beta)\nonumber\\
\dot{v} &=& a \nonumber\\
\dot{\psi} &=& \frac{v}{l_r}\sin(\beta)\nonumber\\
\beta &=& \tan^{-1} \left( \frac{l_r}{l_f+l_r} \tan(\delta_f)\right) \nonumber
\end{eqnarray} 


where
\begin{align}
x &= \text{ global x CoG coordinate} \nonumber \\
y &= \text{ global y CoG coordinate} \nonumber \\
v &= \text{ speed of the vehicle} \nonumber \\
\psi &= \text{ global heading angle} \nonumber \\
\beta &= \text{ angle of the current velocity with respect to the longitudinal axis of the car} \nonumber \\
a &= \text{ acceleration of the center of mass into this direction} \nonumber \\
l_r &= \text{ distance from the center of mass of the vehicle to the rear axle} \nonumber \\
l_f &= \text{ distance from the center of mass of the vehicle to the front axle} \nonumber \\
\delta_f &= \text{ steering angle of the front wheels with respect to the longitudinal axis of the car} \nonumber
\end{align}

Collect the states in one vector $z = [x, y, v, \psi]^T$, and the inputs as $u = [a, \beta]^T$. Obtain a discrete-time model by using Forward Euler Discretization with sampling time $\Delta t = 0.2$. Use $l_f=l_r = 1.738$ in the simulation. You are asked to formulate and solve a parking problem as a finite-time optimal control problem. The vehicle starts from the initial state $\bar{z}_0 = [\hspace{0.5mm}0, \hspace{1mm}3,\hspace{1mm} 0,\hspace{1mm} 0\hspace{0.5mm}]^T$. Our goal is to park the vehicle in the terminal state $\bar{z}_N=[0,0,0,-\pi/2]^T$. Set the horizon $N = 70$ and the state constraints to be: $[-20, \hspace{2.5mm}-5,\hspace{2.5mm}-10, \hspace{2.5mm}-2\pi]^T \leq z(k) \leq [20, \hspace{2.5mm}10,\hspace{2.5mm}10, \hspace{2.5mm}2\pi]^T $. Solve the finite time optimal control problem and plot the results.

$\textbf{Part (a)}$

Parking Problem Formulation 1

Consider the finite time optimal control problem defined below. Formulate and solve the optimzation problem in $\texttt{pyomo}$. 

\begin{align}
\min_{z_0,\ldots,z_N,u_0,\ldots,u_{N-1}} & \sum_{k=N-2}^{N} \|z_k  - \bar{z}_N \|_2^2 \nonumber\\
& z_{k+1} = z_k+f(z_k,u_k)\Delta t &&~\forall k = \left\{0,\ldots,N-1 \right\} \nonumber\\
& z_{min} \leq z_k \leq z_{max} &&~\forall k = \left\{0,\ldots,N \right\} \nonumber\\
& u_{min} \leq u_k \leq u_{max} &&~\forall k = \left\{0,\ldots,N-1 \right\} \nonumber\\
& | \beta_{k+1} - \beta_k | \leq \beta_{d} &&~\forall k = \left\{0,\ldots,N-2 \right\} \nonumber\\
%& |a(k+1) - a(k)| \leq a_d &&~\forall k = \left\{0,\ldots,H_p-1 \right\} \nonumber\\
& z_0 = \bar{z}_0\nonumber\nonumber\\
& z_N = \bar{z}_N\nonumber
\end{align}
Consider the following constraints:

1. The difference of current and previous steering commands are bounded by $\pm 0.2 $ rad. (i.e. $\beta_d = 0.2$)
2. The accelerations are bounded by $|a(k)| \leq 0.3 \ \text{m/s}^2$.
3. The steering control inputs are limited to $|\beta(k)| \leq 0.6$ rad.

### <font color=red> Delivarable 1a: write your answer in the code cell below.</font>

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np 
import pyomo.environ as pyo 

Ts = .2 
N = 70
TFinal = Ts * N

z0 = np.array([0,3,0,0])
zN = np.array([0,0,0,-np.pi/2])
zMax = np.array([20,10,10,2*np.pi])
zMin = np.array([-20,-5,-10,-2*np.pi])
umin = [-0.3,-0.6]
umax = [0.3,0.6]

nz = 4
nu = 2 
l_r = 1.738 

model = pyo.ConcreteModel()
model.tidx = pyo.Set(initialize = range(N+1))
model.zidx = pyo.Set(initialize = range(nz))
model.uidx = pyo.Set(initialize = range(nu))

model.z = pyo.Var(m.zidx,m.tidx)
model.u = pyo.Var(m.uidx,m.tidx)

model.cost = pyo.Objective(
    expr = sum((m.z[i,t] - zN[i])**2 for i in m.zidx for t in m.tidx 
    if t > N-3 and t<N+1),
    sense = pyo.minimize
)

model.c11 = pyo.Constraint(
    model.tidx, rule = lambda m,t:
    model.z[0,t+1] == m.z[0,t] + Ts * (m.z[2,t]* pyo.cos(m.z[3,t]+m.u[1,t]))
    if t < N else pyo.Constraint.Skip
)
model.c12 = pyo.Constraint(
    model.tidx, rule = lambda m,t:
    model.z[1,t+1] == m.z[1,t] + Ts * (pyo.sin(m.z[3,t]+m.u[1,t]))
    if t < N else pyo.Constraint.Skip
)
model.c13 = pyo.Constraint(
    model.tidx, rule = lambda m,t:
    model.z[2,t+1] == m.z[2,t] +Ts* m.u[0,t]
    if t < N else pyo.Constraint.Skip
) 
model.c14 = pyo.Constraint(
    m.tidx, rule = lambda m,t:
    m.z[3,t+1] == m.z[3,t] + Ts * (m.z[2,t]/l_r * pyo.sin(m.u[1,t]))
    if t < N else pyo.Constraint.Skip
)
# zmin <= zk <= zmax 
m.c21 = pyo.Constraint(
    m.zidx,m.tidx, rule = lambda m,i,t:
    m.z[i,t] <= zMax[i]
    if t< N else pyo.Constraint.Skip
)
m.c22 = pyo.Constraint(
    m.zidx,m.tidx, rule = lambda m,i,t:
    m.z[i,t] >= zMin[i]
    if t< N else pyo.Constraint.Skip
)
# umin <= uk <= umax
m.c31 = pyo.Constraint(
    m.uidx, m.tidx, rule = lambda m,i,t:
    m.u[i,t] <= umax[i]
    if t <N else pyo.Constraint.Skip
)
m.c32 = pyo.Constraint(
    m.uidx, m.tidx, rule = lambda m,i,t:
    m.u[i,t] >= umin[i]
    if t <N else pyo.Constraint.Skip
)
# |beta_k+1 - beta_k| <= beta_d
m.c41 = pyo.Constraint(
    m.tidx, rule = lambda m,t:
    m.u[1,t+1] - m.u[1,t] <= 0.2 
    if t < N-1 else pyo.Constraint.Skip
)
m.c41 = pyo.Constraint(
    m.tidx, rule = lambda m,t:
    m.u[1,t+1] - m.u[1,t] >= -0.2 
    if t < N-1 else pyo.Constraint.Skip
)

m.c5 = pyo.Constraint(
    m.zidx, rule = lambda m,i:
    m.z[i,0] == z0[i]
)
m.c6 = pyo.Constraint(
    m.zidx, rule = lambda m,i:
    m.z[i,N] == zN[i]
)


results = pyo.SolverFactory('ipopt').solve(m).write()



    'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 1197
  Number of variables: 424
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termin

$\textbf{Part (b)}$

Parking Problem Formulation 2

Use formulation 1 and add the additional constraints on the difference between current and previous acceleration to get a smoother parking maneuver:

\begin{align}
|a(k+1) - a(k)| \leq a_d \quad \forall k = {0,\ldots,N-2},
\end{align}

where $a_d = 0.06$ m/s$^2$. Formulate the optimzation problem in $\texttt{pyomo}$. 

Show by using plots that you get a smoother control action.

### <font color=red> Delivarable 1b: write your answer in the code cell below.</font>

In [ ]:
# Write your code here:

grid(color='r', linestyle='-', linewidth=2)
plt.figure()
plt.plot(tGrid, x_actual[0,:], 'b')
plt.plot(tGrid, xDesired, 'g')
plt.plot(tGrid, x1, '--r')
plt.legend(['Actual', 'Desired', 'Open-Loop'])
plt.xlabel('Time')
plt.ylabel('x1 Trajectory')
plt.show()

***

# <font color = blue> 2. Unconstrained Linear Finite Time Optimal Control - Batch </font>

Consider the discrete-time dynamic system with the following state space representation:

\begin{align}
 \label{eq:Dyn_Sys}
    \begin{bmatrix}
      x_1(k+1) \\
      x_2(k+1) \end{bmatrix} = \begin{bmatrix}
      0.77 & -0.35 \\
      0.49 & 0.91
    \end{bmatrix}
    \begin{bmatrix}
      x_1 (k) \\
      x_2 (k)
    \end{bmatrix} + \begin{bmatrix}
      0.04 \\
      0.15 
    \end{bmatrix} u(k) 
\end{align}

We want to design a linear quadratic optimal control for this system with a finite horizon $N = 50$. We set the following cost matrices:

\begin{align}
Q = \left[ \begin{matrix}
500 & 0 \\ 0 & 100  \end{matrix} \right],  \ \ R = 1, \ \
P = \left[ \begin{matrix}
1500 & 0 \\
0 & 100
\end{matrix} \right],
\end{align}
and assume that the initial state is $x(0) = [ 1, -1]^T$;

$\textbf{Part (a)}$

Determine the optimal  set of inputs
\begin{align}
U_0= \left[\begin{matrix}
u_0 \\ u_1 \\ \vdots \\ u_{N-1} \\
\end{matrix}\right]
\end{align}

through the Batch Approach, i.e. by writing the dynamic equations as follows:
\begin{align*}
\left[ \begin{array}{c} x_0 \\ x_1 \\ \vdots \\ \vdots \\ x_N \end{array} \right] & = \left[ \begin{array}{c} I \\ A \\ \vdots \\ \vdots \\ A^N \end{array} \right] x(0)+ \left[ \begin{array}{cccc} 0 & \cdots & \cdots & 0 \\ B & 0 & \cdots & 0 \\ AB & B & \cdots & 0 \\ \vdots & \ddots & \ddots & 0 \\ A^{N-1}B & \cdots & AB & B \end{array} \right] \left[ \begin{array}{c} u_0 \\ u_1 \\ \vdots \\ \vdots \\ u_{N-1} \end{array} \right] \\ & = \mathcal{S}^x x(0) + \mathcal{S}^u U_0,
\end{align*}
and using the formula:
\begin{align*}
U_0^*(x(0)) & = - (\mathcal{S}^{uT}\overline{Q}\mathcal{S}^u + \overline{R})^{-1}\mathcal{S}^{uT}\overline{Q}\mathcal{S}^x x(0)  \, ,
\end{align*}
and calculate the optimal cost $J_0^*(x(0))$:
\begin{align*}
J_0^*(x(0)) & = x(0)^T(\mathcal{S}^{xT}\overline{Q}\mathcal{S}^x - \mathcal{S}^{xT}\overline{Q}\mathcal{S}^u(\mathcal{S}^{uT}\overline{Q}\mathcal{S}^u + \overline{R})^{-1}\mathcal{S}^{uT}\overline{Q}\mathcal{S}^x)x(0)\, .
\end{align*}

Print out $U_0^*(x(0))^T$ and $J_0^*(x(0))$.
*Hint:* To efficiently concatenate the matrices use $\texttt{scipy.linalg.block_diag}$ and $\texttt{numpy.kron}$. 

### <font color=red> Delivarable 2a: write your answer in the code cell below.</font>

In [7]:
import numpy as np
import scipy as sp
from scipy.linalg import block_diag
from numpy.linalg import inv

# The function Sx_Su is given to you. Fill in the blanks to write lqrBatch function that calls Sx_Su function. 
# Then call lqrBatch function for the given problem data to calculate U0_star and J0_star. 

def Sx_Su(A, B, N):
    
    nX = np.size(A,0)
    nU = np.size(B,1)

    Sx = np.eye(nX)
    A_tmp = A
    for i in range(N):
        Sx = np.vstack((Sx, A_tmp))
        A_tmp = A_tmp @ A
        
    SxB = Sx @ B
    Su = np.zeros((nX*(N+1),nU*N))
    
    for j in range(N):
        Su_tmp = np.vstack((np.zeros((nX, nU)), SxB[:-nX,:]))
        Su[:, j] = Su_tmp.reshape(Su_tmp.shape[0], )
        SxB = Su_tmp
    
    return Sx, Su

# Fill in the blanks:

def lqrBatch(A,B,Q,R,PN,N):
    Sx, Su = Sx_Su(A, B, N)
    Qbar =  block_diag(np.kron(np.eye(N),Q),PN)
    Rbar = np.kron(np.eye(N),R)
    QSu = Qbar @ Su
    H = Su.T @ QSu + Rbar
    F = Sx.T @ QSu
    K = -inv(H) @ F.T 
    P0 = F@K + Sx.T @ Qbar @ Sx
    return K,P0

# Solve the finite-horizon, LQR problem for a time-invariant discrete-time
# system.  A is nX-by-nX, B is nX-by-nU, (Q,R,PN) are symmetric and positive
# semidefinite (R positive-definite), and of dimension nX-by-nX, nU-by-nU,
# and nX-by-nX, respectively.   N denotes the number of time-steps.  The
# output argument K is N*nU-by-nX, so that K*x0 is the (vertically
# concatenated) sequence of optimal inputs, {u_0, u_1, ..., u_{N-1}}.  The
# optimal cost, from any initial condition x0, is x0'*P0*x0.

# Problem data
A = np.array([[0.77, -0.35],
              [0.49, 0.91]])
B = np.array([[0.04],
              [0.15]])
Q = np.diag((500,100))
R = 1
PN = np.diag((1500,100))
x0 = np.array([1,-1])
N = 5

# call lqrBatch function and calculate U0_star and J0_star
K,P0 = lqrBatch(A,B,Q,R,PN,N)
U0_star = K @ x0    # sequence of optimal inputs
J0_star = x0.T @ P0 @ x0    # the optimal cost

print('K= ', K)
print('P0= ',P0)
print('U0_star = ',U0_star)
print('J0_star = ',J0_star)


K=  [[-2.03227264 -6.14516735]
 [-2.51745006  1.25826235]
 [-1.49314732  1.47593134]
 [-0.79280627  0.9213694 ]
 [-0.70590212  0.88342561]]
P0=  [[ 842.67961988 -299.18321182]
 [-299.18321182  433.23464536]]
U0_star =  [ 4.11289471 -3.77571241 -2.96907866 -1.71417567 -1.58932773]
J0_star =  1874.2806888890866


$\textbf{Part (b)}$

Verify the results of the previous point by solving a numerical optimization problem.
In fact, the cost can be written as a function of  $U_0$  as follows:

\begin{align*}
J_0(x(0),U_0) & = (\mathcal{S}^x x(0) + \mathcal{S}^u U_0)^T\overline{Q}(\mathcal{S}^x x(0) + \mathcal{S}^u U_0) + U_0^T\overline{R}U_0 \\
    & = U_0^T H U_0 + 2 x(0)^TFU_0 + x(0)^T \mathcal{S}^{xT}\overline{Q}\mathcal{S}^x x(0),
\end{align*}

where $H := \mathcal{S}^{uT}\overline{Q}\mathcal{S}^u + \overline{R}$ and $F := \mathcal{S}^{xT}\overline{Q}\mathcal{S}^u$,
and then minimized by solving a quadratic minimization problem.
Check that the optimizer $U_0^*$ and the optimum $J_0(x(0),U_0^*)$ correspond to the ones determined analytically in the previous point. 

*Note:* Make sure you compute the unconstrained solution here and do not have any linear constraints. Use $\texttt{cvxopt.solvers.qp}$ to solve the unconstrained quadratic program. 

### <font color=red> Delivarable 2b: write your answer in the code cell below.</font>

In [8]:
# use cvxopt to solve quadratic program 
# H and F are given and no constraint is included
import cvxopt
from cvxopt import matrix, solvers

# Fill in the blanks:

Qbar = block_diag(np.kron(np.eye(N),Q),PN)
Rbar = np.kron(np.eye(N),R)
Sx, Su = Sx_Su(A,B,N)
QSu = Qbar @ Su
H = Su.T @ QSu +Rbar
F = Sx.T @ QSu



P = 2*H     # write P and q to solve the quadratic program using cvxopt
q = 2 * x0.T @ F 

P = cvxopt.matrix(P, tc='d')
q = cvxopt.matrix(q.T, tc='d')
sol = cvxopt.solvers.qp(P,q)

print('U0_star =', sol['x'])
print('J0_star =', sol['primal objective'] + x0.T @ Sx.T @ Qbar @ Sx @ x0)

U0_star = [ 4.11e+00]
[-3.78e+00]
[-2.97e+00]
[-1.71e+00]
[-1.59e+00]

J0_star = 1874.2806888890864


***

# <font color = blue> 3. Unconstrained Linear Finite Time Optimal Control - Recursive </font>

$\textbf{Part (a)}$

Design the optimal controller through the recursive approach and determine the optimal state-feedback matrices $F_k$. Start from the Riccati Difference Equations, assuming that $P_N = P$, and compute recursively the $P_k$:
\begin{align}
P_k = A^TP_{k+1}A + Q - A^TP_{k+1}B(B^TP_{k+1}B + R)^{-1} B^T P_{k+1} A, \label{eq:RDE}
\end{align}
and then calculate $F_k$ as a function of $P_{k+1}$:
\begin{align}
F_{k} = -(B^TP_{k+1}B + R)^{-1}B^TP_{k+1}A.
\end{align}

Compare the optimal cost $J_0^*(x(0)) = x(0)^T P_0 x(0)$ with question 2.a and check that they are equal. 

### <font color=red> Delivarable 3a: write your answer in the code cell below.</font>

In [9]:
nX = np.size(A,0)
nU = np.size(B,1)

P = np.zeros((nX,nX,N+1))
F = np.zeros((nU,nX,N))
P[:,:,N] = PN

# Fill in the blanks:

for i in range(N-1,-1,-1):
    P_k1 = P[:,:,i+1]
    F[:,:,i] = -inv(B.T @ P_k1 @ B +R) @ B.T @ P_k1 @ A
    P[:,:,i] = A.T @ P_k1 @A + Q +A.T @P_k1 @B@ F[:,:,i]

Jopt_DP = x0.T @ P[:,:,0] @ x0


print('The optimal cost from the recursive approach is:')
print(Jopt_DP)

The optimal cost from the recursive approach is:
1874.2806888890864


$\textbf{Part (b)}$

We want to understand the effect of model uncertainty when using the  two approaches in simulations (batch and recursive). Hence, let us add an additive process disturbance $D w(k)$ to the right-hand side of dynamic equation, $x(k+1) = Ax(k) + Bu(k) + Dw(k)$. Assume the matrix $D$ to be:
\begin{equation*}
D = \left[ \begin{matrix}
0.1 \\ 0.1
\end{matrix} \right],
\end{equation*}
while the process $w(k)$  a Gaussian white noise, with  mean $m = 0$ and variance $\sigma^2 = 10$.

Consider the simulation length to be equal to $N$ time steps and that the input $u_k$ to the system are defined as follows:
\begin{equation*}
u_k=\begin{cases} U_0[k+1] & \mbox{for Batch Approach} \\ F_k x_k & \mbox{for Recursive Approach},
\end{cases}
\end{equation*}
where $U_0[k]$ is the $k$-th component of the vector $U_0$.

Plot a graph of the state evolution over time. What is the difference in the dynamic evolution? What happens if you modify the variance of the disturbance?

A function $\texttt{sysSim}$ that simulates the dynamic system is given to you. This function outputs the next state $x(k+1)$ based on the current states $x(k)$, inputs $u(k)$ and the distubance $w(k)$.
For the generation of the white noise, use $\texttt{noise = numpy.random.normal((mean, var))}$. 

### <font color=red> Delivarable 3b: write your answer in the code cell below.</font>

In [ ]:
import matplotlib.pyplot as plt

# xNext is the next state x(k+1), xCurr and uCurr are the current state x(k) and input u(k), respectively. 

def sysSim(A, B, D, w, xCurr, uCurr):
    xNext = A@xCurr + B@uCurr + D@w
    return xNext


# Put your code here:
# provide a plot of state trajectory over time for the given variance using both approaches.
# provide the same plot when you double the variance.


***

# <font color=blue> 4. Constrained Finite Time Optimal Control - Sparse vs Dense QP Formulations </font>

Consider CFTOC of a discrete-time double-integrator system:
\begin{align}
\min_{\substack{ u_0,\dots,u_{N-1} \\ x_1,\dots,x_N }} ~&~ x_N^TPx_N + \displaystyle{\sum}_{k=0}^{N-1}  x_k^TQx_k + u_k^TRu_k \\
\text{subject to} ~&~ x_{k+1}=
\begin{bmatrix} 1 & 1 \\  0 & 1 \end{bmatrix}
x_k
+
\begin{bmatrix} 0 \\ 1 \end{bmatrix}
u_k \\
& -1 \leq u_k \leq 1, ~k \in \left\{ 0, \ldots, N-1 \right\} \\
& \begin{bmatrix} -15 \\ -15 \end{bmatrix} \leq x(k) \leq \begin{bmatrix} 15 \\ 15 \end{bmatrix}, ~k \in \left\{ 1, \ldots, N \right\}
\end{align}
where $N = 3$, $P = Q = \mathcal{I}_{2 \times 2}$, $R = 0.1$.

$\textbf{Part (a)}$

Let  $x_0 = [-1, -1]^T$.
Determine the QP $\textbf{dense}$ formulation when you substitute the dynamics in the CFTOC. That is, determine $H, f, c, A, b$ for the following problem:
\begin{align*}
\min_{U_0} ~&~ \frac{1}{2} U_0^T H U_0 + f^T U_0 + c\\
\text{subject to} ~&~ A U_0 \leq b.
\end{align*}

where $U_0 =\left[ u_0^T, u_1^T, \ldots, u_{N-1}^T \right]^T$. Write the code to synthesize these matrices and then compute the solution using $\texttt{cvxopt}$. Note that you can drop the $c$ term when using $\texttt{cvxopt}$. You should NOT have any $Aeq$ or $beq$. Make sure to turn in your code and outputs (synthesized matrices, solver solution and optimal value). How many $0$'s are in $A$? In $H$?

In [ ]:
import time
from ttictoc import tic,toc
tic()
time.sleep(1)
t_dense = toc()
print('Elapsed time:',t_dense)

### <font color=red> Delivarable 4a: write your answer in the code cell below.</font>

In [ ]:
import numpy as np
import scipy as sp
from scipy.linalg import block_diag
from numpy.linalg import inv
import time
from ttictoc import tic,toc

# Define state and controller matricies
Ax = np.array([[1, 1],
               [0, 1]])
Bx = np.array([[0],
              [1]])
Q = np.eye(2)
P = np.eye(2)
R = 0.1
N = 3
x0 = np.array([[-1],[-1]])

# Define state and input constraints
ULlim = -1
UUlim = 1
xLlim = np.array([-15, -15])
xUlim = np.array([15, 15])

Sx, Su = Sx_Su(Ax, Bx, N)  
    
Qbar  = ...
Rbar = ...
QSu = ...
H = ...
F = ...
K = ...
P0 = ... 

c = x0.T@Sx.T@Qbar@Sx@x0

A = np.concatenate([np.kron(np.array([[1],[-1]]),np.eye(3)), Su, -Su], axis = 0)
b = np.concatenate([np.ones((nX*N,1)), 15*np.ones((2*nX*(N+1),1))-np.concatenate([Sx,-Sx], axis = 0)@x0], axis = 0)


P = ...    # define P and q to write the quaratic program in cvxopt
q = ...

P = cvxopt.matrix(P, tc='d')
q = cvxopt.matrix(q.T, tc='d')
G = cvxopt.matrix(A, tc='d')
h = cvxopt.matrix(b, tc='d')

tic()
sol = cvxopt.solvers.qp(P,q,G,h)
t_dense = toc()

Uopt_dense = sol['x'] 
Jopt_dense = sol['primal objective']+c

print('The optimal control sequence=', Uopt_dense)
print('The optimal cost=', Jopt_dense)
print('The H matrix=', H)
print('The F matrix=', F)
print('The c matrix=', c)
print('The A matrix=', A)
print('The b matrix:', b)
print('Number of zeros in A matrix=', np.size(A,0)*np.size(A,1) - np.count_nonzero(A))
print('Number of zeros in H matrix=', np.size(H,0)*np.size(H,1) - np.count_nonzero(H))

$\textbf{Part (b)}$

Let  $x_0 = [-1, -1]^T$.
Determine the QP $\textbf{sparse}$ formulation when you do NOT substitute the dynamics in the CFTOC. That is, what are $H, f, A, b, Aeq, beq$ for the following problem:
\begin{align*}
\min_{z} ~&~ \frac{1}{2} z^T H z + f^T z \\
\text{subject to} ~&~ Az \leq b \\
&~ A_{eq}z = b_{eq}.
\end{align*}

where $z = [x_1^T, \ldots, x_N^T, u_0^T, \ldots, u_{N-1}^T]^T$. This requires using $Aeq$ and $beq$. Again, write a script to synthesize these matrices and compute the solution using $\texttt{cvxopt}$. Make sure to turn in the outputs (synthesized matrices, $\texttt{cvxopt.solvers.qp}$ solution and optimal value).  How many $0$'s are in $A$? In $H$? In $Aeq$?

$\textbf{Part (c)}$

Compare the $\texttt{cvxopt.solvers.qp}$ solver times compare using $\texttt{tic toc}$ commands. Check your solutions to make sure the two methods get the same answer.

### <font color=red> Delivarable 4b&c: write your answer in the code cell below.</font>

In [ ]:
Q = np.eye(2)
P = np.eye(2)
R = 0.1
N = 3
x0 = np.array([[-1],[-1]])

H = sp.linalg.block_diag(np.kron(np.eye(N-1),Q),P,np.kron(np.eye(N),R))

Aeq = np.concatenate([np.concatenate([np.eye(2), np.zeros((2,2)), np.zeros((2,2)), -Bx, np.zeros((2,1)), np.zeros((2,1))], axis = 1),
       np.concatenate([-Ax, np.eye(2), np.zeros((2,2)), np.zeros((2,1)), -Bx, np.zeros((2,1))], axis = 1),
       np.concatenate([np.zeros((2,2)), -Ax, np.eye(2), np.zeros((2,1)), np.zeros((2,1)), -Bx], axis = 1)], axis = 0)

beq = np.concatenate([Ax, np.zeros((2,2)), np.zeros((2,2))], axis = 0)@x0

# Fill in the blanks:

A = ...
b = ...

P = ...
q = ...
c = ...

P = cvxopt.matrix(P, tc='d')
q = cvxopt.matrix(q, tc='d')
G = cvxopt.matrix(A, tc='d')
h = cvxopt.matrix(b, tc='d')
Aeq = cvxopt.matrix(Aeq, tc='d')
beq = cvxopt.matrix(beq, tc='d')

tic()
sol = cvxopt.solvers.qp(P,q,G,h,Aeq,beq)
t_sparse = toc()

Uopt_sparse = sol['x'] 
Jopt_sparse = sol['primal objective']+c
Uopt_sparse_array = np.asarray(Uopt_sparse)

print('Uopt_sparse=', sol['x'])
print('Jopt_sparse=', Jopt_sparse)


print('The optimal control sequence=', np.asarray(Uopt_sparse_array[6:]))
print('The optimal cost=', Jopt_sparse)
print('The H matrix=', H)
print('The F matrix=', q)
print('The A matrix=', A)
print('The b matrix=', b)
print('Number of zeros in A matrix=', np.size(A,0)*np.size(A,1) - np.count_nonzero(A))
print('Number of zeros in H matrix=', np.size(H,0)*np.size(H,1) - np.count_nonzero(H))


print('time to solve the dense formulation=', t_dense)
print('time to solve the sparse formulation=', t_sparse)


# <font color=blue> 5. KKT Conditions II </font>

Write a function $\texttt{NLPkkt1}$, which solves the optimization problem defined below, with the following function declaration line, then call the function and print the output,

`def NLPkkt1(z0):
    return kktsat`

where the input $\texttt{z0}$ is the initial guess for your optimizer. The function should output a single logical variable that reflects whether all KKT conditions are satisfied for the constrained nonlinear problem (i.e., 1 stands for all KKT conditions satisfied).

\begin{align}
\min_{z_1,z_2}~ & 3\sin(-2\pi z_1)+2z_1+4+\cos(2\pi z_2)+z_2 \nonumber\\
\text{s.t. } &-1 \leq z_1 \leq 1 \nonumber\\
&-1 \leq z_2 \leq 1 \nonumber
\end{align}

### <font color=red> Delivarable 5: write your answer in the code cell below.</font>

In [12]:
# Write your code here:
def LPQPkkt1(z0 =[]):
  threshold = 1e-5
  model = pyo.ConcreteModel()
  model.z1 = pyo.Var()
  model.z2 = pyo.Var()
  
  model.obj = pyo.Objective(expr = (model.z1)**2 + (model.z2)**2)
  model.Constraint1 = pyo.Constraint(expr = model.z1 <= -3)
  model.Constraint2 = pyo.Constraint(expr = model.z2 <= 4)
  model.Constraint3 = pyo.Constraint(expr = 0 >= 4*model.z1 + 3*model.z2)

  model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
  solver = pyo.SolverFactory('ipopt')
  results = solver.solve(model)

  if results.solver.termination_condition != TerminationCondition.optimal:
    KKTsat = False
  else:
    A = np.array([[1,0],[-1,0],[0,1],[0,-1]])
    #A = np.array([1,0,0,1,4,3]).reshape((3, 2))
    b =  np.array([1,1,1,1])
    zOpt = np.array([pyo.value(model.z1), pyo.value(model.z2)])
    
    u = []
    for c in model.component_objects(pyo.Constraint, active=True):
        print ("Constraint", c)
        for index in c:
            u.append(-model.dual[c[index]]) # The duals in pyomo are defined as -u<=0, so we add a negative sign.
            print(model.dual[c[index]])
    u = np.asarray(u)
    for i in range(len(u)):
        if (u[i] < threshold) & (u[i] > -threshold):
            u[i] = 0 
            
# Checking KKT Conditions: 
    flag_primal = np.all(A@zOpt <= b + threshold)   #  A@zOpt <= b primal feasibility 
    flag_dual = np.all(u >= 0)     # dual feasibility 
    flag_cs = np.all(np.multiply(u,(A@zOpt-b)) < threshold) & np.all(np.multiply(u,(A@zOpt-b)) > -threshold)  # complementary slackness u1*g1 = 0, u2*g2=0
    grad_lagrangian = [-6*np.pi*np.cos(-2*np.pi*zOpt[0])+2,-2*np.pi*np.sin(2*np.pi*zOpt[1])+1] + u.T@A
    
    for i in range(len(grad_lagrangian)):
        if (grad_lagrangian[i] < threshold) & (grad_lagrangian[i] > -threshold):  # gradient of Lagragian evaluated at optimizer point must be zero.    
            grad_lagrangian[i] = 0
    flag_grad = np.all(grad_lagrangian == 0)
    KKT_conditions = np.array([flag_primal, flag_dual, flag_cs, flag_grad])
    if all(KKT_conditions == 1):
        KKTsat = True
    else:
        KKTsat = False  
    print(KKT_conditions)
    return KKTsat

z0 = [1, 2]
print(bool(LPQPkkt1(z0)))  

NameError: ignored